In [11]:
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabaseChain
from urllib.parse import quote  
from sqlalchemy import create_engine, select, Table, MetaData, Column, String
from dotenv import load_dotenv
import os

load_dotenv()

True

# Generating SQL query equivalent to query in natural language through ChatGPT with the help of Langchain

langchain automates the process of creating a prompt for a database schema. To answer queries against an SQL database, the langchain uses the SQLDatabaseChain, whose main task is to provide GPT with information from custom databases.

### Conectando ao Oracle Database

https://analytix.nl/post/querying-complex-database-schemas-with-gpt-and-langchain/
https://python.langchain.com/docs/modules/chains/popular/sqlite
https://python.langchain.com/docs/modules/agents/toolkits/sql_database


In [13]:
username = str(os.getenv('DB_USER_NAME'))
passwd = str(os.getenv('DB_PASS'))
hostname = str(os.getenv('DB_HOST'))
port = str(os.getenv('DB_PORT'))
database = str(os.getenv('DB_NAME'))
sqldriver = str(os.getenv('SQL_DRIVER'))
servicename = str(os.getenv('SERVICE_NAME'))


# dsn = """(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={hostname})
#     (PORT={port}))(CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME={servicename})))"""
    
import cx_Oracle
dsnStr = cx_Oracle.makedsn(host = hostname, port = port, service_name = servicename).replace("SID", "SERVICE_NAME")
    
uri = "{}://{}:{}@{}".format(sqldriver, username, quote(passwd), dsnStr)
print(uri)
db = SQLDatabase.from_uri(uri)

oracle+oracledb://mondial_sql:mondial_sql@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=beatriz.tecgraf.puc-rio.br)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=orcl.tecgraf.puc-rio.br)))


Testando a conexão

In [3]:
db.get_table_names()

c:\Users\wendyzv\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\sql_database.py:279: UserWarning: This method is deprecated - please use `get_usable_table_names`.
  warnings.warn(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'dr$i_mondial_tmdc_test$i',
 'dr$i_mondial_tmdc_test$k',
 'dr$i_mondial_tmdc_test$n',
 'dr$i_mondial_tmdc_test$r',
 'dr$i_mondial_tmdc_test$u',
 'dr$i_mondial_tmdp_test$i',
 'dr$i_mondial_tmdp_test$k',
 'dr$i_mondial_tmdp_test$n',
 'dr$i_mondial_tmdp_test$r',
 'dr$i_mondial_tmdp_test$u',
 'dr$i_mondial_tpv_test$i',
 'dr$i_mondial_tpv_test$k',
 'dr$i_mondial_tpv_test$n',
 'dr$i_mondial_tpv_test$r',
 'dr$i_mondial_tpv_test$u',
 'dr$idx_ctx_00d49b9c00d8b7c23$i',
 'dr$idx_ctx_00d49b9c00d8b7c23$k',
 'dr$idx_ctx_00d49b9c00d8b7c23$n',
 'dr$idx_ctx_00d49b9c00d8b7c23$r',
 'dr$idx_ctx_00d49b9c00d8b7c23$u',
 'dr$idx_ctx_011be7ab6d6a4a465$i',
 'dr$idx_ctx_011be7ab6d6a4a465$k',
 'dr$idx_ctx_011be7ab6d6a4a465$n',
 'dr$idx_ctx_011be7ab6d6a4a465$r',
 'dr$idx_ctx_011be7ab6d6a4a465$u',
 'dr$idx_ctx_0dea24ec14ddda69e$i',
 'dr$idx

Após importar e rodar localmente, gerou erros,
- o SQLAlchemy não sabia que tipo era GEOCOORD

- o número de tokens enviados ao modelo referente a informações do banco de dados foi maior que o limite máximo de tokens(4097).


Então a próxima abordagem foi diminuir o tamanho da base de dados, adicionando 
2 linhas de exemplos para cada tabela

In [4]:
include_tables = ["airport", "borders", "city", "citylocalname", "cityothername", "citypops", "continent", "country", 
                  "countrylocalname", "countryothername", "countrypops", "economy", "encompasses", "ethnicgroup", "organization", 
                  "politics", "population", "province", "provincelocalname", "provinceothername", "provpops", "religion"]

db = SQLDatabase.from_uri(uri, include_tables=include_tables, sample_rows_in_table_info=2)

db.get_table_names()

['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'economy',
 'encompasses',
 'ethnicgroup',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion']

Verificando o prompt que o langchain passa para o GPT

In [5]:
print(db.table_info)
f = open("prompt_generated_by_langchain.txt", "w")
f.write(db.table_info)
f.close()



CREATE TABLE airport (
	iatacode VARCHAR(3 CHAR) NOT NULL, 
	name VARCHAR(100 CHAR), 
	country VARCHAR(4 CHAR), 
	city VARCHAR(50 CHAR), 
	province VARCHAR(50 CHAR), 
	island VARCHAR(50 CHAR), 
	latitude NUMBER, 
	longitude NUMBER, 
	elevation NUMBER, 
	gmtoffset NUMBER, 
	CONSTRAINT sys_c00114043 PRIMARY KEY (iatacode), 
	CONSTRAINT airplat CHECK ((Latitude >= -90) AND (Latitude <= 90)), 
	CONSTRAINT airplon CHECK ((Longitude >= -180) AND (Longitude <= 180))
)

/*
2 rows from airport table:
iatacode	name	country	city	province	island	latitude	longitude	elevation	gmtoffset
KMG	Wujiaba	CN	Kunming	Yunnan	None	24.992364	102.743536	1895.0	8.0
TSN	Binhai	CN	Tianjin	Tianjin	None	39.124353	117.346183	3.0	8.0
*/


CREATE TABLE borders (
	country1 VARCHAR(4 CHAR) NOT NULL, 
	country2 VARCHAR(4 CHAR) NOT NULL, 
	length NUMBER, 
	CONSTRAINT borderkey PRIMARY KEY (country1, country2), 
	CONSTRAINT sys_c00113990 CHECK (Length > 0)
)

/*
2 rows from borders table:
country1	country2	length
AL	GR	282.

In [6]:
db_chain = SQLDatabaseChain(llm=ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo'), database=db, verbose=True)

c:\Users\wendyzv\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\sql_database\base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [7]:
def execute_query(query):
    try:
        db_chain.run(query)
    except:
        print("\n Consulta SQL gerada, mas houve erro ao executá-la")

In [8]:
execute_query("Country limit 10")



> Entering new  chain...
Country limit 10
SQLQuery:SELECT "name"
FROM country
FETCH FIRST 10 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [9]:
execute_query("Country with largest area")



> Entering new  chain...
Country with largest area
SQLQuery:SELECT "name" 
FROM country 
ORDER BY area DESC 
FETCH FIRST 1 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [10]:
execute_query("What province is the city of Bandar Seri Begawan in?")



> Entering new  chain...
What province is the city of Bandar Seri Begawan in?
SQLQuery:SELECT province
FROM city
WHERE name = 'Bandar Seri Begawan';
 Consulta SQL gerada, mas houve erro ao executá-la


In [11]:
execute_query("What is relationship between India and Mauritius Island?")




> Entering new  chain...
What is relationship between India and Mauritius Island?
SQLQuery:SELECT a.country, b.country, a.island
FROM airport a
JOIN airport b ON a.island = b.island
WHERE a.country = 'IN' AND b.country = 'MU'
SQLResult: []
Answer:There is no relationship between India and Mauritius Island in terms of airports.
> Finished chain.


In [12]:
execute_query("What percentage of religious people are hindu in India?")



> Entering new  chain...
What percentage of religious people are hindu in India?
SQLQuery:SELECT percentage
FROM religion
WHERE country = 'IN' AND name = 'Hindu'
SQLResult: []
Answer:Final answer here: There are no records in the religion table for the country 'IN' and the religion 'Hindu'.
> Finished chain.


In [13]:
execute_query("Tell me five brazilians cities")



> Entering new  chain...
Tell me five brazilians cities
SQLQuery:SELECT "name" 
FROM city 
WHERE country = 'BR' 
FETCH FIRST 5 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [14]:

execute_query("tell me percentage of religious by religion in india")



> Entering new  chain...
tell me percentage of religious by religion in india
SQLQuery:SELECT r.name, r.percentage
FROM religion r
WHERE r.country = 'IND'
SQLResult: [('Hindu', Decimal('79.8')), ('Muslim', Decimal('14.2')), ('Catholic', Decimal('1.5')), ('Protestant', Decimal('0.6')), ('Christian Orthodox', Decimal('0.4')), ('Sikh', Decimal('1.7')), ('Buddhist', Decimal('0.7')), ('Jains', Decimal('0.4')), ('Sarnaism', Decimal('0.7'))]
Answer:The percentage of religious by religion in India is as follows:
- Hindu: 79.8%
- Muslim: 14.2%
- Catholic: 1.5%
- Protestant: 0.6%
- Christian Orthodox: 0.4%
- Sikh: 1.7%
- Buddhist: 0.7%
- Jains: 0.4%
- Sarnaism: 0.7%
> Finished chain.


In [15]:
execute_query("tell me total percentage of religious people who are Hindu by continent")



> Entering new  chain...
tell me total percentage of religious people who are Hindu by continent
SQLQuery:SELECT c.name, SUM(r.percentage) AS total_percentage
FROM continent c
JOIN encompasses e ON c.name = e.continent
JOIN country co ON e.country = co.code
JOIN religion r ON co.code = r.country
WHERE r.name = 'Hindu'
GROUP BY c.name
SQLResult: [('Asia', Decimal('261.5')), ('Europe', Decimal('6.6')), ('Australia/Oceania', Decimal('33.4')), ('Africa', Decimal('59.7')), ('South America', Decimal('47.1')), ('North America', Decimal('33.4'))]
Answer:The total percentage of religious people who are Hindu by continent is as follows:
- Asia: 261.5%
- Europe: 6.6%
- Australia/Oceania: 33.4%
- Africa: 59.7%
- South America: 47.1%
- North America: 33.4%
> Finished chain.


In [16]:
#Essa consulta funcionou no software cliente do oracle, mas não funcionou aqui
execute_query("tell me airports with elevation more than 1000")



> Entering new  chain...
tell me airports with elevation more than 1000
SQLQuery:SELECT "iatacode", "name", "elevation" 
FROM airport 
WHERE "elevation" > 1000
ORDER BY "elevation" DESC
FETCH FIRST 5 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


## Conclusões 

Tive problemas com tipos de colunas customizadas do banco, no caso o Geocoord, pois o SqlAlchemy não o reconhece. Pode ser contornado nesse [link](https://docs.sqlalchemy.org/en/20/core/custom_types.html#sqlalchemy.types.UserDefinedType), mas não consegui implementar. 

Então importei o banco para o local e alterei as colunas com tipo Geocoord para Varchar(50) e assim, consegui contrornar. Ao criar o SQLDatabaseChain e executar uma consulta tive erro pois o prompt gerado pelo langchain passava  do limite máximo de tokens. 

Em sua documentação, o langchain oferece a possibilidade de incluir e excluir quais tabelas queremos. Então, retornei e fiz a comunicação com o banco "mondial_sql" do servidor e fiz as consultas. Gerava o SQL, mas dava DatabaseError e não retornava os resultados. E para verificar e validar se o SQL gerado trazia resultados, a consulta gerada pelo GPT era passada diretamente em um software cliente SGBD  para ser executada.

Algumas consultas SQL geradas estavam retornando as colunas com aspas e dava erro na execução da consulta, tanto aqui quanto no software cliente.
```
SELECT "name" 
FROM country 
ORDER BY area DESC 
FETCH FIRST 1 ROWS ONLY;
```
Quando removi elas, a consulta foi executada corretamente no software cliente.

As consultas SQL geradas que não possuiam aspas em algumas das colunas executavam normalmente no software cliente, mas aqui estava dando erro.

Nessa consulta "What is relationship between India and Mauritius Island?" o sql gerado buscou uma relação entre esses dois lugares pela coluna island da tabela airport (na abordagem passando o prompt foi por Borders). Mas não houve resultados.

Interessante, que nessa consulta "tell me total percentage of religious people who are Hindu by continent" trouxe resultados corretos onde envolvia 3 joins e agregação. O danke não respondeu correto para as seguintes palavras chaves:
- "percentage total hindu continent" "total percentage hindu continent"
- "percentage sum hindu continent" "sum total hindu continent"

Testes

In [24]:
execute_query("What are the provinces with an area greater than 1000?")



> Entering new  chain...
What are the provinces with an area greater than 1000?
SQLQuery:SELECT "name" 
FROM province 
WHERE "area" > 1000
 Consulta SQL gerada, mas houve erro ao executá-la


In [23]:
execute_query("What are the provinces with an area more than 1000?")



> Entering new  chain...
What are the provinces with an area more than 1000?
SQLQuery:SELECT "name" 
FROM province 
WHERE "area" > 1000
 Consulta SQL gerada, mas houve erro ao executá-la


In [15]:
execute_query("Tell me about cities of Cuba")



> Entering new  chain...
Tell me about cities of Cuba
SQLQuery:SELECT name, country, province, population, latitude, longitude, elevation
FROM city
WHERE country = 'CU'
SQLResult: []
Answer:There are no cities in Cuba in the database.
> Finished chain.


In [16]:
execute_query("city of Cuba")



> Entering new  chain...
city of Cuba
SQLQuery:SELECT name
FROM city
WHERE country = 'CUB'
SQLResult: []
Answer:Final answer: There are no cities in Cuba in the database.
> Finished chain.


In [25]:
execute_query("Say me about the total of city of cuba?")



> Entering new  chain...
Say me about the total of city of cuba?
SQLQuery:SELECT COUNT(*) AS total_cities
FROM city
WHERE country = 'CUB';
 Consulta SQL gerada, mas houve erro ao executá-la


In [26]:
execute_query("Say me about the total of Cities of cuba?")



> Entering new  chain...
Say me about the total of Cities of cuba?
SQLQuery:SELECT COUNT(*) AS total_cities
FROM city
WHERE country = 'CUB';
 Consulta SQL gerada, mas houve erro ao executá-la


In [27]:
execute_query("Say me about the total of cities of cuba?")



> Entering new  chain...
Say me about the total of cities of cuba?
SQLQuery:SELECT COUNT(*) AS total_cities
FROM city
WHERE country = 'CUB';
 Consulta SQL gerada, mas houve erro ao executá-la


In [20]:
execute_query("Tell me the largest area.")



> Entering new  chain...
Tell me the largest area.
SQLQuery:SELECT MAX(area) AS largest_area
FROM country
SQLResult: [(17075200,)]
Answer:The largest area is 17,075,200.
> Finished chain.


In [28]:
execute_query("Areas different from 100")



> Entering new  chain...
Areas different from 100
SQLQuery:SELECT name, area
FROM country
WHERE area <> 100
ORDER BY area DESC
FETCH FIRST 5 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [39]:
execute_query("Shows me cities with longitude between 0 and 6")



> Entering new  chain...
Shows me cities with longitude between 0 and 6
SQLQuery:SELECT "name", "country", "province", "latitude", "longitude"
FROM city
WHERE "longitude" BETWEEN 0 AND 6
 Consulta SQL gerada, mas houve erro ao executá-la


In [30]:
execute_query("Shows me Airports with elevation more than 5000")



> Entering new  chain...
Shows me Airports with elevation more than 5000
SQLQuery:SELECT "iatacode", "name", "elevation"
FROM airport
WHERE "elevation" > 5000
 Consulta SQL gerada, mas houve erro ao executá-la


In [31]:
execute_query("What language is spoken in Poland?")



> Entering new  chain...
What language is spoken in Poland?
SQLQuery:SELECT localname
FROM countrylocalname
WHERE country = 'PL'
SQLResult: [('Rzeczpospolita Polska',)]
Answer:The language spoken in Poland is "Rzeczpospolita Polska".
> Finished chain.


In [32]:
execute_query("Tell me about the religion that is practiced in Haiti")



> Entering new  chain...
Tell me about the religion that is practiced in Haiti
SQLQuery:SELECT name
FROM religion
WHERE country = 'HT'
SQLResult: []
Answer:Final answer here: There is no information available about the religion practiced in Haiti in the database.
> Finished chain.


In [33]:
execute_query("How deep is Lake Kariba?")



> Entering new  chain...
How deep is Lake Kariba?
SQLQuery:SELECT elevation
FROM lake
WHERE name = 'Kariba'
SQLResult: []
Answer:There is no data available for the elevation of Lake Kariba in the given tables.
> Finished chain.


In [34]:
execute_query("What is the capital of Georgia?")



> Entering new  chain...
What is the capital of Georgia?
SQLQuery:SELECT capital
FROM country
WHERE name = 'Georgia'
SQLResult: [('Tbilisi',)]
Answer:Tbilisi
> Finished chain.


In [35]:
execute_query("What is the total of provinces of Netherlands?")



> Entering new  chain...
What is the total of provinces of Netherlands?
SQLQuery:SELECT COUNT(*) AS total_provinces
FROM province
WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Total provinces of Netherlands is 12.
> Finished chain.


In [36]:
execute_query("total provinces Netherland")



> Entering new  chain...
total provinces Netherland
SQLQuery:SELECT COUNT(*) AS total_provinces
FROM province
WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Total provinces in Netherlands is 12.
> Finished chain.


In [37]:
execute_query("total Province Netherlands")



> Entering new  chain...
total Province Netherlands
SQLQuery:SELECT COUNT(*) AS total_provinces
FROM province
WHERE country = 'NL'
SQLResult: [(12,)]
Answer:Total provinces in Netherlands: 12
> Finished chain.


In [38]:
execute_query("What percentage of religious people are hindu in thailand?")



> Entering new  chain...
What percentage of religious people are hindu in thailand?
SQLQuery:SELECT percentage
FROM religion
WHERE country = 'TH' AND name = 'Hindu'
SQLResult: []
Answer:Final answer here: There are no records of Hindu religious people in Thailand.
> Finished chain.


In [40]:
execute_query("What are the provinces with an area greater than 1000 in Niger?")



> Entering new  chain...
What are the provinces with an area greater than 1000 in Niger?
SQLQuery:SELECT name
FROM province
WHERE country = 'NE' AND area > 1000
SQLResult: []
Answer:There are no provinces with an area greater than 1000 in Niger.
> Finished chain.


In [53]:
execute_query("The Chatanga River belongs to which country?")



> Entering new  chain...
The Chatanga River belongs to which country?
SQLQuery:SELECT country
FROM borders
WHERE country2 = 'Chatanga'
 Consulta SQL gerada, mas houve erro ao executá-la


In [43]:
execute_query("What province is the city of Glendale in?")



> Entering new  chain...
What province is the city of Glendale in?
SQLQuery:SELECT province
FROM city
WHERE name = 'Glendale'
SQLResult: [('Arizona',), ('California',)]
Answer:The city of Glendale is in the provinces of Arizona and California.
> Finished chain.


In [54]:
execute_query("What is the area of the city Granada in Nicaragua?")



> Entering new  chain...
What is the area of the city Granada in Nicaragua?
SQLQuery:SELECT area
FROM city
WHERE name = 'Granada' AND country = 'NI';
 Consulta SQL gerada, mas houve erro ao executá-la


In [45]:
execute_query("tell me about the Caribbean economic")



> Entering new  chain...
tell me about the Caribbean economic
SQLQuery:SELECT *
FROM economy
WHERE country IN (
    SELECT country
    FROM encompasses
    WHERE continent = 'Caribbean'
)
SQLResult: []
Answer:There is no information available about the Caribbean economy in the database.
> Finished chain.


In [55]:
execute_query("Tell me about the country with the area largest.")



> Entering new  chain...
Tell me about the country with the area largest.
SQLQuery:SELECT name, area
FROM country
ORDER BY area DESC
FETCH FIRST 1 ROWS ONLY;
 Consulta SQL gerada, mas houve erro ao executá-la


In [50]:
execute_query("What is the host province of the International Energy Agency?")



> Entering new  chain...
What is the host province of the International Energy Agency?
SQLQuery:SELECT province
FROM organization
WHERE name = 'International Energy Agency'
SQLResult: [('ÃŽle-de-France',)]
Answer:ÃŽle-de-France
> Finished chain.


In [56]:
execute_query(" What is Mexico's independence day?")



> Entering new  chain...
 What is Mexico's independence day?
SQLQuery:SELECT independence
FROM politics
WHERE country = 'MEX';
 Consulta SQL gerada, mas houve erro ao executá-la
